## Question 1: Morphing
#### abstract
In this question, we implement image morphing for two face images (with the same size ) in following steps:
<div>
<ol>
<li>
Find corresponding points between images
<li>
Apply Delaunay triangulation algorithm for points in the first image
and get triangles
<li>
For each morph image, fill each triagle with a linear combination of corresponding
triangles in first and last image, using Affine transform and warping
</ol>
</div>
Functions used in the question are implemented in `q1_funcs.py`, and main code is in
`q1.py`.

### `q1_funcs.py`
Following libraries are used in this file :

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from scipy.spatial import Delaunay


First function is used to read points from file. Corresponding points between
images are written in files `leo.txt` and `tom.txt`.


In [ ]:
def read_points(file):
    f = open(file)
    n = int(f.readline())
    points = []
    for line in f.readlines():
        point = line.split(" ")
        [x, y] = int(point[0]), int(point[1])
        points.append([x, y])
    return n, points



Next function generates and inserts 8 additional points, four corners
of the image and midpoint of sides of the rectangle containing the image

In [ ]:
def generate_additional_points(img):
    h, w = img.shape[0], img.shape[1]
    h2, w2 = h // 2, w // 2
    points = [[0, 0], [0, h2], [0, h - 1], [w2, h - 1], [w - 1, h - 1], [w - 1, h2], [w - 1, 0], [w2, 0]]
    return points

Function `delaunay_triangulation` simply uses Delaunay library in scipy
to perform triangulations

In [ ]:
def delaunay_triangulation(points):
    triangles = Delaunay(points)
    return triangles

`find_morphed_triangle_points` gets as input, selected points in first and last image and a coefficient alpha,
and returns a new list which contains corresponding points in the morphed image

In [ ]:
def find_morphed_triangle_points(points1, points2, alpha):
    result = []
    for i in range(points1.shape[0]):
        x = int((1 - alpha) * points1[i, 0] + alpha * points2[i, 0])
        y = int((1 - alpha) * points1[i, 1] + alpha * points2[i, 1])
        result.append([x, y])
    return np.asarray(result)


Function `morph_triangles` takes a source image, a triangle in source image and a
triangle in destination image as input, computes the Affine transform taking first triangle to second ,
and warps the source. It also returns a mask of destination triangle

In [ ]:
def morph_triangles(img, tri_src, tri_dst):
    mask = np.zeros(img.shape, dtype='uint8')
    M = cv2.getAffineTransform(tri_src, tri_dst)
    dest = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]), None, flags=cv2.INTER_LINEAR,
                          borderMode=cv2.BORDER_REFLECT_101)
    cv2.fillConvexPoly(mask, tri_dst.astype('int'), (1.0, 1.0, 1.0), 16, 0)
    result = dest
    return result, mask


`generate_morphed_image` takes two images, their corresponding points, a triangulation on the first image and
a coefficient alpha as input, and creates a morphed image from given images with ration alpha

In [ ]:
def generate_morphed_image(img1, img2, points1, points2, triangles, alpha):
    mean_points = find_morphed_triangle_points(points1, points2, alpha)
    tri1 = triangles.simplices
    result = np.zeros(img1.shape, dtype='uint8')
    for tri in tri1:
        tri_points1 = np.asarray([points1[tri[0]], points1[tri[1]], points1[tri[2]]], dtype='float32')
        tri_points2 = np.asarray([points2[tri[0]], points2[tri[1]], points2[tri[2]]], dtype='float32')
        tri_points_morph = np.asarray([mean_points[tri[0]], mean_points[tri[1]], mean_points[tri[2]]], dtype='float32')

        result1, mask = morph_triangles(img1, tri_points1, tri_points_morph)
        result2, mask = morph_triangles(img2, tri_points2, tri_points_morph)
        result = (result * (1 - mask) + (alpha * result2 + (1 - alpha) * result1) * mask).astype('uint8')

    return result


Final function `generate_morphing_list` takes two image, their corresponding points and number of frames
as input and returns a list containing the morphed images between given images

In [ ]:
def generate_morphing_list(img1, img2, points1, points2, num_of_frames):
    result = []
    triangles = delaunay_triangulation(points1)
    delta, alpha = 1.0 / num_of_frames, 0.0
    while alpha < 1:
        img_morph = generate_morphed_image(img1, img2, points1, points2, triangles, alpha)
        result.append(img_morph)
        alpha += delta
    return result


Function `generate_video` creates a video from a given list of images

In [ ]:
def generate_video(results, output_path, size, fps):
    video = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for img in results:
        video.write(img)
    video.release()

### q1.py
Our main file simply uses above functions to perform morphing between two images

In [ ]:
import numpy as np
import cv2
import q1_funcs as funcs

img1 = cv2.imread("images/res01.jpg")
img2 = cv2.imread("images/res02.jpg")

''' reading points'''
n, points1 = funcs.read_points("images/tom.txt")
n, points2 = funcs.read_points("images/leo.txt")

''' extending points '''
points1.extend(funcs.generate_additional_points(img1))
points2.extend(funcs.generate_additional_points(img2))

points1 = np.asarray(points1, dtype='int')
points2 = np.asarray(points2, dtype='int')

''' creating final result '''
result = []
images = funcs.generate_morphing_list(img1, img2, points1, points2, 45)
cv2.imwrite("images/res03.jpg", images[15])
cv2.imwrite("images/res04.jpg", images[30])
result.extend(images)
images.reverse()
result.extend(images)
funcs.generate_video(result, "images/morph.mp4", (img1.shape[1], img2.shape[0]), 30)
